In [11]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import json
import time
import random
import re

In [29]:
# WebDriver 설정
options = webdriver.ChromeOptions()
options.add_argument("window-size=1920x1080")
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument(
    "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
    "AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
)
driver = webdriver.Chrome(service=webdriver.chrome.service.Service(ChromeDriverManager().install()), options=options)

# 스크롤 로직
def scroll_to_bottom(scrollable_element):
    last_height = driver.execute_script("return arguments[0].scrollHeight", scrollable_element)
    while True:
        driver.execute_script("arguments[0].scrollTop += 600;", scrollable_element)
        time.sleep(1)  # 페이지 로딩 대기
        new_height = driver.execute_script("return arguments[0].scrollHeight", scrollable_element)
        if new_height == last_height:
            break
        last_height = new_height

# 네이버 지도 접속
driver.get("https://map.naver.com/")
time.sleep(3)

# 검색어 입력 및 검색
search_box = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.CLASS_NAME, "input_search"))
)
search_box.send_keys("서대문구 맛집")
search_box.send_keys(Keys.ENTER)
time.sleep(5)

# iframe 전환
iframe = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.CSS_SELECTOR, "iframe#searchIframe"))
)
driver.switch_to.frame(iframe)

# 결과 리스트 초기화
results = []
visited_urls = set()
MAX_RESULTS = 200
page_number = 1  # 페이지 번호 초기화

while len(results) < MAX_RESULTS:
    try:
        # 스크롤을 맨 아래까지 내리기
        scrollable_element = driver.find_element(By.CLASS_NAME, "Ryr1F")
        scroll_to_bottom(scrollable_element)

        # 현재 페이지의 가게 리스트 가져오기
        store_elements = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CLASS_NAME, "place_bluelink"))
        )
        print(f"현재 {page_number}페이지) 총 {len(store_elements)}개의 가게를 찾았습니다.")

        for store in store_elements:
            try:
                store.click()
                time.sleep(random.uniform(3, 5))

                # 상세 iframe 전환
                driver.switch_to.default_content()
                detail_iframe = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "iframe#entryIframe"))
                )
                driver.switch_to.frame(detail_iframe)

                # 페이지 소스 파싱
                html = driver.page_source
                soup = BeautifulSoup(html, "lxml")

                # 가게 정보 추출
                place_name = soup.select_one("span.GHAhO").text.strip() if soup.select_one("span.GHAhO") else "N/A"
                category = soup.select_one("span.lnJFt").text.strip() if soup.select_one("span.lnJFt") else "N/A"
                address = soup.select_one("span.LDgIH").text.strip() if soup.select_one("span.LDgIH") else "N/A"
                current_url = driver.current_url
                res_code = re.findall(r"place/(\d+)", current_url)
                store_url = f'https://pcmap.place.naver.com/restaurant/{res_code[0]}/review/visitor#' if res_code else "N/A"

                # 중복 여부 확인
                if store_url not in visited_urls:
                    visited_urls.add(store_url)
                    results.append({
                        "가게 이름": place_name,
                        "업종구분": category,
                        "주소": address,
                        "URL": store_url
                    })
                    print(f"{len(results)}. {place_name}")

                # iframe 복귀
                driver.switch_to.default_content()
                driver.switch_to.frame(iframe)

                if len(results) >= MAX_RESULTS:
                    print("최대 200개 결과를 수집했습니다. 크롤링을 종료합니다.")
                    break
            except Exception as e:
                print(f"가게 크롤링 중 오류 발생: {e}")

        # '다음' 버튼 클릭
        try:
            next_button = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, '//a[@class="eUT2V" and not(@aria-disabled="true")]'))
            )
            driver.execute_script("arguments[0].click();", next_button)
            print(f"'다음' 버튼 클릭: 페이지 {page_number + 1}로 이동 중")
            time.sleep(random.uniform(5, 10))
        except Exception as e:
            print(f"'다음' 버튼 클릭 중 오류 발생: {e}")
            break

        # 페이지 번호 증가
        page_number += 1

    except Exception as e:
        print(f"페이지 크롤링 중 오류 발생: {e}")
        break

# JSON 파일 저장
file_name = "seodaemun_restaurant_200.json"
with open(file_name, "w", encoding="utf-8") as file:
    json.dump(results, file, ensure_ascii=False, indent=4)

print(f"크롤링 완료. 결과는 {file_name}에 저장되었습니다.")
driver.quit()

현재 1페이지) 총 76개의 가게를 찾았습니다.
1. 역전포장마차 신촌점
2. 헤일우드
3. 정육면체 신촌점
4. 봉일천 장군집
5. 머노까머나 신촌점
6. 스웨이커피스테이션
7. 작은스페인
8. 이석덕생면파스타 신촌점
9. 한사발포차 신촌점
10. 목란
11. 설레임삼겹살 신촌점
12. 오리지날 이대본점
13. 연경 연희점
14. 백순대본가새맛 신촌연대점
15. 아민 이화
16. 시오 연희동본점
17. 만동제과 연남점
18. 이빠네마 그릴
19. 청년밥상문간 이화여자대학교점
20. 하남돼지집 서대문역점
21. 몽주방 신촌점
22. 치즈웨이브
23. 돗돗
24. 카페 모래내 1번지
25. 담산 신촌본점
26. 파이홀
27. 한옥집김치찜 본점
28. 수빈
29. 마우디
30. 끝집 가재울본점
31. 한술식당 신촌점
32. 쿳사 연희
33. 산천마루
34. 강남불백 신촌점
35. 녹원쌈밥 연희점
36. 화로상회 신촌점
37. 수지상회 신촌점
38. 라구식당
39. 가야가야
40. 쟁반집8292 신촌점
41. 센트그릴
42. 아건 이대본점
43. 유우 연희동본점
44. 혼신꼬치 신촌점
45. 쿠치
46. 소문난집
47. 칠백식당
48. 월순철판동태찜
49. 비아37 신촌본점
50. 담솥 신촌점
51. 당나발포차 신촌점
52. 돈불1971
53. 다대기실비 서대문역
54. 포가레 신촌점
55. 오르랔베이커리
56. 목구멍 신촌점
57. 고래고래신촌점
58. 카라멘야
59. 타코몽
60. 매뉴팩트커피 연희 본점
61. 마더린러 베이글
62. 피터팬1978
63. 고삼이 신촌점
64. 오향만두
65. 화우 서대문역본점
66. 하늘초밥
67. 하나마토 신촌
68. 티앙팡 오후의 홍차
69. 평안도식당 신촌점
70. 야바이
71. 남영동양문 서대문역점
72. 포티드
73. 처음본명품민물장어 홍제점
74. 푸어링아웃
75. 카츠업
76. 복성각
'다음' 버튼 클릭 중 오류 발생: Message: 
Stacktrace:
	GetHandleVerifier [0x004633E3+2